**Step 1: Import Required Libraries**

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier

**Step 2: Load the Dataset**

In [4]:
term_deposit_df = pd.read_csv("/content/bank-full.csv")

**Step 3: Encode Target and Categorical Variables**

In [5]:
term_deposit_df['y'] = term_deposit_df['y'].str.lower().map({'yes': 1, 'no': 0})
term_deposit_df['marital'] = term_deposit_df['marital'].str.lower().map({'married': 1, 'single': 2, 'divorced': 3})

**Step 4: Create Age Groups**

In [6]:
bins = [0, 18, 60, 100]
group_nums = [1, 2, 3]
term_deposit_df['AgeGroup'] = pd.cut(term_deposit_df['age'], bins, labels=group_nums)

**Step 5: Handle Missing Values**

In [7]:
def replace_nan_with_mean_mode(df):
    for column in df.columns:
        if df[column].dtype in ['int64', 'float64']:
            df[column].fillna(df[column].mode()[0], inplace=True)
        else:
            df[column].fillna(df[column].mode()[0], inplace=True)
    return df

term_deposit_df = replace_nan_with_mean_mode(term_deposit_df)

/tmp/ipython-input-1794111918.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(df[column].mode()[0], inplace=True)
/tmp/ipython-input-1794111918.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', t

**Step 6: Handle Outliers in balance**

In [8]:
def replace_outliers_with_median(df, column_name):
    Q1 = df[column_name].quantile(0.25)
    Q3 = df[column_name].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    median = df[column_name].median()
    df.loc[(df[column_name] < lower_bound) | (df[column_name] > upper_bound), column_name] = median
    return df

term_deposit_df = replace_outliers_with_median(term_deposit_df, 'balance')


**Step 7: Clip Extreme Values in duration**

In [9]:
q_low = term_deposit_df['duration'].quantile(0.01)
q_hi = term_deposit_df['duration'].quantile(0.99)
term_deposit_df['duration'] = term_deposit_df['duration'].clip(lower=q_low, upper=q_hi)

**Step 8: Drop Unnecessary Columns**

In [10]:
term_deposit_df.drop(columns=['pdays', 'previous'], inplace=True)

**Step 9: Split Data into Features and Target**

In [11]:
X = term_deposit_df.drop('y', axis=1)
y = term_deposit_df['y']

**Step 10: Encode Categorical Variables (One-Hot)**

In [12]:
X = pd.get_dummies(X)

**Step 11: Train-Test Split**

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Step 12: Feature Scaling**

In [14]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

**Step 13: Train Random Forest Model**

In [15]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

**Step 14: Model Evaluation**

In [16]:
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9037929890523057
Confusion Matrix:
 [[7712  240]
 [ 630  461]]
Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.97      0.95      7952
           1       0.66      0.42      0.51      1091

    accuracy                           0.90      9043
   macro avg       0.79      0.70      0.73      9043
weighted avg       0.89      0.90      0.89      9043

